# Vespaを利用したサンプル検索

Vespaを利用した検索などを試すためのノートブックです。

## 接続先の設定

**VESPAHOST**に接続先を設定します。サンプルでは、ローカルネットワーク環境にあるVespaに接続しています。
接続先の各種設定については公式ドキュメントをご覧ください。
https://pyvespa.readthedocs.io/en/latest/getting-started-pyvespa.html

**NAMESPACE**が接続先のネームスペース名、**DOCTYPE**がドキュメントタイプ名です。

In [46]:
from vespa.application import Vespa

VESPAHOST = "http://vespa:8080"
NAMESPACE = "product"
DOCTYPE = "product"

client = Vespa(url=VESPAHOST)

## TODO スキーマの確認

対象となるインデックスのスキーマを確認します。
スキーマはファイルから確認します。

スキーマファイルは[../../schema/vespa/schemas/product.sd](../../schema/vespa/schemas/product.sd)になります。

## データの確認

件数を確認する。

* `select * from {DOCTYPE}` : 取得するドキュメントタイプの指定
* `where true` : 全件取得する条件（where句がないとエラーとなる）
* `limit 0` : ヒット件数を知りたいので検索結果は取得しない

レスポンスの"fields.totalCount"が件数です。

In [47]:
import json
body = {
    "yql": f"select * from {DOCTYPE} where true limit 0"
}
response = client.query(body=body)
print(json.dumps(response.get_json(), indent=2))

{
  "root": {
    "id": "toplevel",
    "relevance": 1.0,
    "fields": {
      "totalCount": 339053
    },
    "coverage": {
      "coverage": 100,
      "documents": 339053,
      "full": true,
      "nodes": 1,
      "results": 1,
      "resultsFull": 1
    }
  }
}



検索してみる。

In [48]:
import pandas as pd

body = {
    "yql": f"select * from {DOCTYPE} where true limit 10"
}
response = client.query(body=body)
df = pd.DataFrame(d["fields"] for d in response.get_json()["root"]["children"])
df

,sddocname,documentid,product_id,product_title,product_locale,type,product_brand,product_color
0,product,id:product:product::0008431809,0008431809,i-SPY On a Car Journey in France: Spy it! Scor...,jp,book,NaN,NaN
1,product,id:product:product::0134685717,0134685717,Feedback Control of Dynamic Systems (What's Ne...,jp,book,Pearson,NaN
2,product,id:product:product::0008431825,0008431825,i-SPY London: Spy it! Score it! (Collins Miche...,jp,book,NaN,NaN
3,product,id:product:product::0141182679,0141182679,On the Road: Penguin UK Edition (Penguin Moder...,jp,book,Penguin Classics,NaN
4,product,id:product:product::0091894506,0091894506,Who Moved My Cheese For Teens,jp,book,NaN,NaN
5,product,id:product:product::0140447334,0140447334,Plays: Ivanov; The Seagull; Uncle Vanya; Three...,jp,book,Penguin Classics,Black
6,product,id:product:product::0062976583,0062976583,"The Boy, the Mole, the Fox and the Horse",jp,book,HarperOne,NaN
7,product,id:product:product::0007420277,0007420277,The Phantom of the Opera (Collins Classics),jp,book,NaN,NaN
8,product,id:product:product::0307930211,0307930211,A Very Busy Sticker Book (The World of Eric Ca...,jp,book,NaN,NaN
9,product,id:product:product::0262038900,0262038900,Spotify Teardown: Inside the Black Box of Stre...,jp,book,The MIT Press,Black


## クエリを指定して検索

In [49]:
body = {
    "yql": f'select * from {DOCTYPE} where product_color contains "ブラック" limit 10'
}
response = client.query(body=body)
#print(response.get_json())
print(f"{response.get_json()['root']['fields']['totalCount']}件ヒット")
df = pd.DataFrame(d["fields"] for d in response.get_json()["root"]["children"])
df

23893件ヒット


,sddocname,documentid,product_id,product_title,product_bullet_point,product_brand,product_color,product_locale,type,image,product_description
0,product,id:product:product::8883701038,8883701038,"Caderno Clássico, Preto, Capa Dura, Sem Pauta,...",192 PÁGINAS\nCAPA DURA\nPAPEL MARFIM ACID-FREE...,Moleskine(モレスキン),ブラック,jp,book,NaN,NaN
1,product,id:product:product::B00002N6SL,B00002N6SL,MAG-LITE(マグライト) 2AA BP BK M2A016V,燭光2200cp\nランプ寿命3~4回(電池を交換する回数)\n電池寿命/連続点灯6~7時間...,MAG-LITE(マグライト),ブラック,jp,product,https://m.media-amazon.com/images/I/81qbd9XyLJ...,NaN
2,product,id:product:product::B00004OCKO,B00004OCKO,OXO 軽量 柔らかグリップ ワインオープナー,<b>メーカー型番:</b> 31781\n<b>サイズ:</b> 幅7.5×長さ20×厚み...,OXO (オクソー),ブラック,jp,product,https://m.media-amazon.com/images/I/711CfjjgrL...,NaN
3,product,id:product:product::B00004SD7B,B00004SD7B,FISKARS (フィスカース) Fiskars/フィスカース 7854 X25 ガーデンア...,28-Inch,FISKARS (フィスカース),ブラック,jp,product,https://m.media-amazon.com/images/I/419bsJdbXu...,NaN
4,product,id:product:product::B0000536M2,B0000536M2,MAYBELLINE FULL N' SOFT MASCARA THICK & HEALTH...,MASCARA | FULL N' SOFT MASCARA THICK & HEALTHY,MAYBELLINE,ブラック,jp,product,https://m.media-amazon.com/images/I/71SG3xJHMN...,MASCARA | FULL N' SOFT MASCARA THICK & HEALTHY
5,product,id:product:product::B000068O1A,B000068O1A,Hosa CPP-201 1m 両側モノラルフォンプラグ×2 オーディオケーブル,ホサ\n世界的なケーブルメーカー『Hosa（ホサ）』のオーディオケーブルです。\n【特長】・...,Hosa,ブラック,jp,product,https://m.media-amazon.com/images/I/814XTim7ND...,NaN
6,product,id:product:product::B000068O1N,B000068O1N,Hosa STP-201 1m ステレオフォンオス-モノラルフォンオス×2 インサーションケーブル,ホサ\nコネクタにステレオフォンオス（1/4” TRS）とモノラルフォンオス（1/4” TS...,Hosa,ブラック,jp,product,https://m.media-amazon.com/images/I/31rkpsoWgn...,NaN
7,product,id:product:product::B000068O3F,B000068O3F,Hosa CMP-105 1.5m ステレオミニプラグ-モノラルフォンプラグ オーディオケーブル,ホサ\n世界的なケーブルメーカー『Hosa（ホサ）』のオーディオケーブルです。\nステレオミ...,Hosa,ブラック,jp,product,https://m.media-amazon.com/images/I/71eTICqQu4...,NaN
8,product,id:product:product::B000068O3D,B000068O3D,Hosa CMP-110 3m ステレオミニプラグ-モノラルフォンプラグ オーディオケーブル,ホサ\n世界的なケーブルメーカー『Hosa（ホサ）』のオーディオケーブルです。\nステレオミ...,Hosa,ブラック,jp,product,https://m.media-amazon.com/images/I/21tvFi8u1J...,NaN
9,product,id:product:product::B00006JSUF,B00006JSUF,LODGE(ロッジ) ロジック10.25インチキッチンダッチオーブン L8DO3 (並行輸入品),NaN,ロッジ(Lodge),ブラック,jp,product,https://m.media-amazon.com/images/I/51iSDLDmt2...,NaN


## [TODO]ベクトル検索（knn検索）

`text`の文字列をLLMのモデルを使って、ベクトルに変換し、knn検索を実行する。
インデックスにはすでに同じLLMのモデルを使用してドキュメントごとにベクトルが付与されているものとする。
